In [1]:
%pip list | grep google-cloud-aiplatform
%pip list | grep google-api-core

Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

vertexai.init(project='adsp-capstone-once-upon', location = 'us-central1')

In [3]:
model = GenerativeModel("gemini-1.0-pro")
import re
from transformers import pipeline

# Initialize the text summarization pipeline
summarizer = pipeline("summarization")

def split_sentences(text):
    # Split the text into sentences using regex
    return re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", text)

def summarize_text(text):
    # Use the summarization pipeline to generate a summary of the text
    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    return summary

def generate_summary(input_prompt, max_tokens = 4000, characters = None):
    input_text = input_prompt
    # Generate response for the current chapter
    response = model.generate_content(
        [input_text],
        generation_config={
            "max_output_tokens": max_tokens,
            "temperature": 0.9,
            "top_p": 1
        }
    )
    generated_text = response.text
    print(generated_text)
    return generated_text

def generate_chapter(input_summary, chapter_number, max_tokens=7000, characters=None):
    # Combine the summary and the last sentences to form the input prompt
    input_text = f"{input_summary}"
    
    # Generate response for the current chapter
    response = model.generate_content(
        [input_text],
        generation_config={
            "max_output_tokens": max_tokens,
            "temperature": 0.9,
            "top_p": 1
        }
    )
    generated_text = response.text
    print(f"Chapter {chapter_number}:")
    print(generated_text)
    return generated_text

def extract_characters(input_story):
    input_text = f"From this story, identify all the characters and their associated personality traits and actions associated with each character in English: {input_story}. \n Give me the response in this format: \n Character Name: <name of character> \n Personality: <list of characteristics separated by comma> \n Actions: <List of actions this character is likely to perform (general)>"
    response = model.generate_content(
        [input_text],
        generation_config={
            "temperature": 0.9,
            "top_p": 1
        }
    )
    generated_text = response.text
    #print("Characters and personality: ", generated_text)
    return generated_text

c:\Users\Christian\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
import time

def generate_story_in_chapters(summary_object, chapter1_structure, chapter2_structure, chapter3_structure):
    
    start_time = time.time()
    
    summary_input = f"This is the outline for this 3-chapter story: {summary_object} /n Based on the title and outline, generate a LENGTHY first chapter of ATLEAST 1000 WORD TOKENS of this story in PARAGRAPHS (not bullet points) fit for a 5 year old kid in English. This chapter should have ATLEAST 1000 words. DO NOT display any additional notes, just the story that the audience should read, and match word length requirements. Overall, no. of tokens should be less than 8000 tokens (all characters included)."
    #print("\n---- END OF CHAPTERS BACKBONE  ----\n")
    
    # Generate the first chapter
    chapter1 = generate_chapter(summary_input, 1, max_tokens=8000)  # Using the initial input prompt
    global characters
    characters = extract_characters(chapter1)

    # Summarize chapter1
    chapter1_summary = summarize_text(chapter1)
    # Generate subsequent chapters, extending the narrative
    sentences = split_sentences(chapter1)
    last_sentences = " ".join(sentences[-5:]) if sentences else ""
    
    #prompt input for chapter 2
    chapter_1_summary = f"Second chapter title and summary: {chapter2_structure} /n Previous Chapter 1 Summary: {chapter1_summary} \n Characters and their associated personalities and behaviour so far: {characters}"
    instructions_2 = f"Taking this second chapter title and outline, previous chapter summary, and character information provided into consideration, generate a LENGTHY second chapter of ATLEAST 1000 WORD TOKENS of this story in PARAGRAPHS (not bullet points) fit for a 5 year old kid in English. This chapter should have ATLEAST 1000 words. DO NOT display any additional notes, just the story that the audience should read, and match word length requirements. Overall, no. of tokens should be less than 8000 tokens (all characters included)."
    final_input_2 = f"{chapter_1_summary} \n \n {instructions_2}"
    chapter2 = generate_chapter(final_input_2, 2, max_tokens=8000)

    # Summarize chapters 1 and 2
    chapters12_summary = summarize_text(f"{chapter2}")
    sentences = split_sentences(chapter2)
    last_sentences = " ".join(sentences[-5:]) if sentences else ""
    
    #prompt input for chapter 3
    chapter_12_summary = f"Third chapter title and summary: {chapter3_structure} /n Previous two Chapters Summary: {chapters12_summary} \n Characters and their associated personalities and behaviour so far: {characters}"
    instructions_3 = f"Based on this third chapter title and outline, previous two chapters summary and character information provided, generate a LENGTHY third chapter of ATLEAST 1000 WORD TOKENS of this story in PARAGRAPHS (not bullet points) fit for a 5 year old kid in English. This chapter should have ATLEAST 1000 words. DO NOT display any additional notes, just the story that the audience should read, and match word length requirements. Overall, no. of tokens should be less than 8000 tokens (all characters included)."
    final_input_3 = f"{chapter_12_summary} \n \n {instructions_3}"
    chapter3 = generate_chapter(final_input_3, 3, max_tokens=8000)
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total time taken: {total_time} seconds")

In [5]:
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [6]:
# def chat_loop(chat, summary_object: str, user_response: str):
#     while user_response.upper() == "NO":
#         question = input("Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY): ")
#         if question.upper() == "MODIFY":          
#             new_prompt = input("Please provide the changes you want to see in this story!: ")
#             summary_prompt = f"Make the following changes to your new story summary: {new_prompt}"
#             summary_object = get_chat_response(chat, summary_prompt)
#             print(summary_object)
#             user_response = input("Do you want to continue? (YES/NO): ")
#         if question.upper() == "NEW STORY":
#             new_prompt = input("Let's build you a new story! What would you like to hear a story about?")
#             summary_prompt = prompt = f"You will be generating a story of 3 chapters with total narration time of 15 minutes. Give me a summary with brief descriptions for each of the three chapters of this story that is engaging for a 5 year old. The story prompt - {new_prompt}. \n Note: You will be using this chapter wise to generate the actual chapters later on. To ensure the chapters are long enough, give enough information and detail in the summary that can be built up on during story generation."
#             summary_object = get_chat_response(chat, summary_prompt)
#             print(summary_object)
#             user_response = input("Do you want to continue? (YES/NO): ")
#     chapters = re.split(r'Chapter \d+: ', summary_object)[1:]
#     # Separate the chapters into individual strings
#     chapter1_structure = "Chapter 1: " + chapters[0]
#     chapter2_structure = "Chapter 2: " + chapters[1]
#     chapter3_structure = "Chapter 3: " + chapters[2]
#     generate_story_in_chapters(summary_object, chapter1_structure, chapter2_structure, chapter3_structure)

In [6]:
#NEW -- try/except block to regenerate if errors come up

def chat_loop(chat, summary_object: str, user_response: str):
    while user_response.upper() == "NO":
        question = input("Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY): ")
        if question.upper() == "MODIFY":          
            new_prompt = input("Please provide the changes you want to see in this story!: ")
            summary_prompt = f"Make the following changes to your new story summary: {new_prompt}"
            summary_object = get_chat_response(chat, summary_prompt)
            print(summary_object)
            user_response = input("Do you want to continue? (YES/NO): ")
        if question.upper() == "NEW STORY":
            new_prompt = input("Let's build you a new story! What would you like to hear a story about?")
            summary_prompt = prompt = f"You will be generating a story of 3 chapters with total narration time of 15 minutes. Give me a summary with brief descriptions for each of the three chapters of this story that is engaging for a 5 year old. The story prompt - {new_prompt}. \n Note: You will be using this chapter wise to generate the actual chapters later on. To ensure the chapters are long enough, give enough information and detail in the summary that can be built up on during story generation."
            summary_object = get_chat_response(chat, summary_prompt)
            print(summary_object)
            user_response = input("Do you want to continue? (YES/NO): ")
    chapters = re.split(r'Chapter \d+: ', summary_object)[1:]
    # Separate the chapters into individual strings
    chapter1_structure = "Chapter 1: " + chapters[0]
    chapter2_structure = "Chapter 2: " + chapters[1]
    chapter3_structure = "Chapter 3: " + chapters[2]
    try:
        generate_story_in_chapters(summary_object, chapter1_structure, chapter2_structure, chapter3_structure)
    except Exception as e:
        print(e)
        print("\n Oops Sorry! Something went wrong... let me regenrate the story for you! \n")
        chat_loop(chat, summary_object, user_response)

In [7]:
def identify_theme(chat):
    themes = ['Family', 'Friendship', 'Animals', 'School', 'Holidays', 'Princess', 'Superhero', 'Adventure', 'Fantasy', 'Courage', 'Kindness']
    theme_prompt = f"These are the list of genres: {themes}. \n Which of these genre from this list, can this story for which you generated a summary fall into? \n Response Format - Python List of names of the genres in quotes separated by commas."
    theme_classification = get_chat_response(chat, theme_prompt)
    return theme_classification

In [9]:
#user_age = 10 

#USER EXPERIENCE
#input_from_user = "A story about a blind boy named mcqueen who wanted to fly planes"
#prompt = f"You will be generating a story of 3 chapters with total narration time of 15 minutes. Give me a summary with brief descriptions for each of the three chapters of this story that is engaging for a {user_age} year old. The story prompt - {input_from_user}. \n Note: You will be using this chapter wise to generate the actual chapters later on. To ensure the chapters are long enough, give enough information and detail in the summary that can be built up on during story generation. Each chapter's summary should be in a paragrah format."
#print(get_chat_response(chat, prompt))
#summary_object = get_chat_response(chat, prompt)
#print(summary_object)

#user_response = input("Do you want to continue? (YES/NO): ")
#chat_loop(chat, summary_object, user_response)

### ***Blind Boy Becoming Pilot***

In [8]:
user_age = 6

# #USER EXPERIENCE
input_from_user = "A story about a blind boy becoming a pilot."
prompt = f"You will be generating a story of 3 chapters with total narration time of 15 minutes. Give me a summary with brief descriptions for each of the three chapters of this story that is engaging for a {user_age} year old. The story prompt - {input_from_user}. \n Note: You will be using this chapter wise to generate the actual chapters later on. To ensure the chapters are long enough, give enough information and detail in the summary that can be built up on during story generation. Each chapter's summary should be in a paragrah format."
# #print(get_chat_response(chat, prompt))
summary_object = get_chat_response(chat, prompt)
# print(summary_object)

# user_response = input("Do you want to continue? (YES/NO): ")
# chat_loop(chat, summary_object, user_response)

c:\Users\Christian\anaconda3\envs\llm\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Mia - Small town girl becomes successful baker

In [12]:
# user_age = 6

# #USER EXPERIENCE
# input_from_user = "A story Mia, a small town girl, becoming the best chef."
# prompt = f"You will be generating a story of 3 chapters with total narration time of 15 minutes. Give me a summary with brief descriptions for each of the three chapters of this story that is engaging for a {user_age} year old. The story prompt - {input_from_user}. \n Note: You will be using this chapter wise to generate the actual chapters later on. To ensure the chapters are long enough, give enough information and detail in the summary that can be built up on during story generation. Each chapter's summary should be in a paragrah format."
# #print(get_chat_response(chat, prompt))
# summary_object = get_chat_response(chat, prompt)
# print(summary_object)

# user_response = input("Do you want to continue? (YES/NO): ")
# chat_loop(chat, summary_object, user_response)

### ***Sara discovers secret door in her backyard that leads to a magical forest***

In [9]:
user_age = 6

#USER EXPERIENCE
input_from_user = "A little girl named Sara discovers a secret door in her backyard that leads to a magical forest where animals can talk, and she has to help the animals solve a mystery to save their home."
prompt = f"You will be generating a story of 3 chapters with total narration time of 15 minutes. Give me a summary with brief descriptions for each of the three chapters of this story that is engaging for a {user_age} year old. The story prompt - {input_from_user}. \n Note: You will be using this chapter wise to generate the actual chapters later on. To ensure the chapters are long enough, give enough information and detail in the summary that can be built up on during story generation. Each chapter's summary should be in a paragrah format."
#print(get_chat_response(chat, prompt))
summary_object = get_chat_response(chat, prompt)
print(summary_object)

user_response = input("Do you want to continue? (YES/NO): ")
chat_loop(chat, summary_object, user_response)

## Chapter Summaries:

**Chapter 1: The Mysterious Door**

* Sara, a bright and curious little girl, loves exploring her backyard. One sunny afternoon, she notices a small, rusty door hidden beneath the climbing vines.
* Driven by an insatiable curiosity, Sara pushes the door open, revealing a hidden passageway shrouded in soft, colorful light.
* The passage leads to a breathtaking world – a magical forest teeming with vibrant colors, singing flowers, and talking animals. 
*  Sara befriends a wise old owl who tells her the forest is in danger. A mischievous creature called "The Shadow" has stolen the magical spring that gives life to the forest, causing the flowers to wilt and the animals to lose their vitality.
* Determined to help her new friends, Sara sets out on a quest to find the stolen spring and restore the forest's magic. 

**Chapter 2: The Adventures Begin**

* Guided by the owl's cryptic clues, Sara embarks on a journey through the magical forest.
* She encounters friendly t

In [14]:
import textstat